In [2]:
"""Create a data pipeline that every month checks a folder in a Google shared 
drive for any new xlsx spreadsheets and then takes that new data from 
Google Drive to a bucket in Google Cloud Storage, without using service account keys 
(organizational security limitation). I think this is best done with 
cloud functions and cloud scheduler but alternative methods are welcome if justifiable.

After that, create another cloud function that takes any new raw data in 
the Google Storage bucket and processes it in BigQuery, adding the data as 
new rows to a master spreadsheet. The processed data should be stored as a
csv in BigQuery. This should happen monthly as well.

I can provide a dummy spreadsheet of raw data. As long as this pipeline 
functions by bringing the data from Google Drive every month into GCS and then 
transforms and stores it in BigQuery, then this job was successful."""

import os 
import google.auth

from google.oauth2 import service_account

from config import config_env

from function.function import  FromDriveToStorage
import google.auth


CONFIG_ENV = os.getenv("ENV") or "dev"
config = config_env[CONFIG_ENV]

# Create credentials with Drive & BigQuery API scopes.
# Both APIs must be enabled for your project before running this code.
# credentials, project = google.auth.default(
#     scopes=config.SCOPES
# )

key_path='/home/portfolio/GCP-Templates/cloud_function/from_drive_to_storage/function/drive.json'
creds=None
if CONFIG_ENV=="dev":
     creds = service_account.Credentials.from_service_account_file(key_path) # If you're using local machine    
     scoped_credentials = creds.with_scopes(config().SCOPES)    
# else:
# creds, _ = google.auth.default()
# scoped_credentials = creds.with_scopes(config.SCOPES)
# 




TypeError: __init__() got an unexpected keyword argument 'creds'

In [3]:
from __future__ import print_function

import os.path

import google.auth
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# If modifying these scopes, delete the file token.json.

SCOPES=['https://www.googleapis.com/auth/drive',
    'https://www.googleapis.com/auth/drive.file',
    'https://www.googleapis.com/auth/drive.readonly',
    'https://www.googleapis.com/auth/drive.metadata.readonly',
    'https://www.googleapis.com/auth/drive.appdata',
    'https://www.googleapis.com/auth/drive.metadata',
    'https://www.googleapis.com/auth/drive.photos.readonly',
]



"""Shows basic usage of the Drive v3 API.
Prints the names and ids of the first 10 files the user has access to.
"""
credentials = None
# The file token.json stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.
if os.path.exists('token.json'):
    credentials = Credentials.from_authorized_user_file('token.json', SCOPES)
# If there are no (valid) credentials available, let the user log in.
if not credentials or not credentials.valid:
    if credentials and credentials.expired and credentials.refresh_token:
        credentials.refresh(Request())
    else:
        credentials, project = google.auth.default(scopes=SCOPES)
try:
    service = build('drive', 'v3', credentials=credentials)

    # Call the Drive v3 API
    results = service.files().list(
        pageSize=10, fields="nextPageToken, files(id, name)").execute()
    items = results.get('files', [])

    if not items:
        print('No files found.')
        
    print('Files:')
    for item in items:
        print(u'{0} ({1})'.format(item['name'], item['id']))
except HttpError as error:
    # TODO(developer) - Handle errors from drive API.
    print(f'An error occurred: {error}')




/home/gcp-templates/lib/python3.9/site-packages/google/auth/_default.py:83: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


An error occurred: <HttpError 403 when requesting https://www.googleapis.com/drive/v3/files?pageSize=10&fields=nextPageToken%2C+files%28id%2C+name%29&alt=json returned "Request had insufficient authentication scopes.". Details: "[{'message': 'Insufficient Permission', 'domain': 'global', 'reason': 'insufficientPermissions'}]">


In [4]:
import os
from google.oauth2 import service_account
key_path=os.path.dirname(os.path.dirname("credentials") +"drive") +"drive.json"
creds = service_account.Credentials.from_service_account_file(key_path) # If you're using local machine    

SCOPES = ['https://www.googleapis.com/auth/drive.metadata.readonly']
scoped_credentials = creds.with_scopes(SCOPES)    

# service_account_info = json.loads(os.getenv('GDRIVE_AUTH'))
SCOPES = ['https://www.googleapis.com/auth/drive.metadata.readonly']

# creds = service_account.Credentials.from_service_account_info(
#         service_account_info, scopes=SCOPES)

service = build('drive', 'v3', credentials=scoped_credentials)
results = service.files().list(
        pageSize=5, fields="nextPageToken, files(id, name, mimeType, size, parents, modifiedTime)").execute()
    # get the results
items = results.get('files', [])
    # list all 20 files & folders
items

[{'mimeType': 'application/vnd.google-apps.folder',
  'id': '1uymD6k_4JTTP4YuTP_NBTAftS4nteU5o',
  'name': 'germination_analysis',
  'modifiedTime': '2023-03-03T23:34:10.600Z'},
 {'mimeType': 'text/csv',
  'parents': ['1uymD6k_4JTTP4YuTP_NBTAftS4nteU5o'],
  'size': '32036',
  'id': '1kCoIzOE9Sj3xYporr1CEjXn7_AKwrORp',
  'name': 'ABA.csv',
  'modifiedTime': '2023-02-22T21:19:10.000Z'}]

In [6]:
import argparse
import os

from googleapiclient import discovery
from googleapiclient.http import build_http
from oauth2client import client, file, tools

client_secrets = os.path.join(os.path.dirname("drive"), "drive.json")
client_secrets = os.path.join(os.getcwd(), 'credentials.json')
SCOPES = ['https://www.googleapis.com/auth/drive.metadata.readonly']
name='drive'
version="3"

# flags = tools.argparser.parse_args(args=['--noauth_local_webserver'])
flags = argparse.Namespace(auth_host_name='localhost',                                
                                logging_level='ERROR',
                                noauth_local_webserver=True)


# Set up a Flow object to be used if we need to authenticate.
flow = client.flow_from_clientsecrets(
    client_secrets, scope=SCOPES
)

credentials = storage.get()
if credentials is None or credentials.invalid:
    credentials = tools.run_flow(flow, storage, flags)
http = credentials.authorize(http=build_http())

service = discovery.build(name, version, http=http)




Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?client_id=318040672941-jbi4pi3rhckhlcpv5cp32v0c01lofqml.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.metadata.readonly&access_type=offline&response_type=code



AttributeError: 'tuple' object has no attribute 'tb_frame'

In [ ]:
results = service.files().list(
        pageSize=5, fields="nextPageToken, files(id, name, mimeType, size, parents, modifiedTime)").execute()
    # get the results
items = results.get('files', [])
    # list all 20 files & folders
items

In [ ]:
from __future__ import print_function

import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# If modifying these scopes, delete the file token.json.
SCOPES = ["https://www.googleapis.com/auth/drive.readonly"]
key_path = os.path.join(os.getcwd(), 'credentials.json')


creds = None
# The file token.json stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.
if os.path.exists('token.json'):
    creds = Credentials.from_authorized_user_file('token.json', SCOPES)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        # flow = InstalledAppFlow.from_client_secrets_file(
        #     'credentials.json', SCOPES)
        flow = InstalledAppFlow.from_client_secrets_file(
                key_path, SCOPES)

        # creds = flow.run_local_server(port=0)
        name ="a"            
        storage = file.Storage(name + ".dat")
        creds = tools.run_flow(flow, storage)

    # Save the credentials for the next run
    with open('token.json', 'w') as token:
        token.write(creds.to_json())

try:
    service = build('drive', 'v3', credentials=creds)

    # Call the People API
    print('List 10 connection names')
    results = service.files().list(
        pageSize=1, fields="nextPageToken, files(id, name, mimeType, size, parents, modifiedTime)").execute()
    # get the results
    items = results.get('files', [])
    # list all 20 files & folders
    connections = results.get('connections', [])

    for person in connections:
        names = person.get('names', [])
        if names:
            name = names[0].get('displayName')
            print(name)
except HttpError as err:
    print(err)




In [ ]:
# Llama al API para listar los archivos
results = service.files().list(
    pageSize=10, fields="nextPageToken, files(id, name)").execute()
items = results.get('files', [])

# Imprime el resultado
if not items:
    print('No se encontraron archivos.')
else:
    print('Archivos:')
    for item in items:
        print(f'{item["name"]} ({item["id"]})')
